# Fairness in IBM FL: Scikitlearn Logistic Classification

## Our Paper: Mitigating Bias in Federated Learning 

Check it out [here](https://arxiv.org/abs/2012.02447)!

## Outline:
- [Add conda environment to Jupyter Notebook](#setup)
- [FL and Fairness](#intro)
- [Aggregator](#Aggregator)
    - [Aggregator Configuration](#Aggregator-Configuration)
    - [Running the Aggregator](#Running-the-Aggregator)
- [Starting Parties](#Starting-Parties)
- [Training and Evaluation](#Training-and-Evaluation)
- [Visualize Results](#Visualize-Results)
- [Shut Down](#Shut-Down)

## Add conda environment to Jupyter Notebook <a name="setup"></a>

Please ensure that you have activated the `conda` environment following the instructions in the project README.

Once done, run the following commands in your terminal to install your conda environment into the Jupyter Notebook:

1. Once you have activated the conda environment, install the `ipykernel` package: `conda install -c anaconda ipykernel`

2. Next, install the `ipykernel` module within Jupyter Notebook: `python -m ipykernel install --user --name=<conda_env>`

3. Please install the `matplotlib` package for your conda environment.

4. Finally, restart the jupyter notebook once done. Ensure that you are running this Notebook from `<project_path>/examples/keras_classifier`, where project_path is the directory where the IBMFL repository was cloned.

When the Notebook is up and running it may prompt you to choose the kernel. Use the drop down to choose the kernel name same as that chosen when running `conda activate <conda_env>`. If no prompt shows up, you can change the kernel by clicking _Kernel_ > _Change kernel_ > _`<conda_env>`_.

## Federated Learning (FL) and Fairness <a name="intro"></a>

**Federated Learning (FL)** is a distributed machine learning process in which each participant node (or party) retains their data locally and interacts with  other participants via a learning protocol. In this notebook, we demonstrate the adaption and usage of popular **bias mitigation techniques** for FL. We examine bias from the perspective of social fairness, as opposed to contribution fairness.

Bias mitigation approaches in machine learning mainly measure and reduce undesired bias with respect to a *sensitive attribute*, such as *age* or *race*, in the training dataset. 

We utilize [IBM FL](https://github.com/IBM/federated-learning-lib) to have multiple parties train a classifier to predict whether a person in the [Adult dataset](http://archive.ics.uci.edu/ml/datasets/Adult) makes over $50,000 a year. We have adapted 2 centralized fairness methods, Reweighing and Prejudice Remover, into 3 federated learning bias mitigation methods: Local Reweighing, Global Reweighing with Differential Privacy, and Federated Prejudice Removal. With these methods, we can run a variety of fairness experiments in IBM FL.

For a more technical dive into IBM FL, refer the whitepaper [here](https://arxiv.org/pdf/2007.10987.pdf).

## Fairness Techniques <a name="fairness"></a>

[Reweighing](https://link.springer.com/article/10.1007/s10115-011-0463-8) is a centralized pre-processing bias mitigation method, which works primarily by attaching weights to samples in the training dataset. This method accesses the entire training dataset and computes weights as the ratio of the expected probability to the observed probability of the sample, calculated based on the sensitive attribute/label pairing in question. We adapt this centralized method into two federated learning techniques, Local Reweighing and Global Reweighing with Differential Privacy.

**Local reweighing**: To fully protect parties' data privacy, each party computes reweighing weights locally based on its own training dataset during pre-processing and then uses the reweighing dataset for its local training. Therefore, parties do not need to communicate with the aggregator or reveal their sensitive attributes and data sample information.

**Global Reweighing with Differential Privacy**: If parties agree to share sensitive attributes and noisy data statistics, parties can employ this fairness method. During the pre-processing phase, the aggregator will collect statistics such as the noisy number of samples with privileged attribute values, compute global reweighing weights  based on the collected statistics, and share them with parties. By adjusting the amount of noise injected via epsilon, parties can control their data leakage while still mitigating bias. 

[Prejudice Remover](https://github.com/algofairness/fairness-comparison/tree/master/fairness/algorithms/kamishima) is an in-processing bias mitigation method 440 proposed for centralized ML, which works by adding a fairness-aware regularizer to the regular logistic loss function. We adapt this centralized method into Federated Prejudice Remover.

**Federated Prejudice Removal**: Each party applies the Prejudice Remover algorithm to train a less biased local model, and shares only the model parameters with the aggregator. The aggregator can then employ existing FL algorithms, like simple average and FedAvg, etc., to update the global model.

Further details about the algorithms and datasets utilized, as well as experimental setup, are included in our [paper](https://arxiv.org/abs/2012.02447).

## Fairness Metrics <a name="mnist"></a>

In fairness evaluation, there is no single, all-inclusive metric. Literature uses multiple metrics to measure several aspects, painting a composition of fairness. We use four highly-utilized fairness metrics: Statistical Parity Difference, Equal Odds Difference, Average Odds Difference, and Disparate Impact.

**Statistical Parity Difference**: Calculated as the ratio of the success rate between the unprivileged and privileged groups. The ideal value for this metric is 0, and the fairness range is between -0.1 and 0.1, as defined by [AI Fairness 360](https://aif360.mybluemix.net/).

**Equal Odds Difference**: Calculated as the true positive rate difference between the unprivileged and privileged groups. The ideal value for this metric is 0, and the fairness range is between -0.1 and 0.1, similarly defined by AI Fairness 360.

**Average Odds Difference**: Calculated as the mean of the false positive rate difference and the true positive rate difference, both between the unprivileged and privileged groups. The ideal value for this metric is 0, and the fairness range is between -0.1 and 0.1, similarly defined by AI Fairness 360.

**Disparate Impact**: Calculated as the difference of the success rate between the unprivileged and privileged groups. The ideal value for this metric is 1, and the fairness range is between 0.8 and 1.2, similarly defined by AI Fairness 360.

### Getting things ready
We begin by setting the number of parties that will participate in the federated learning run and splitting up the data among them.

In [1]:
import sys
sys.path.append('../..')
import os
os.chdir("../..")

num_parties = 2  ## number of participating parties
dataset = 'adult'

We use `examples/generate_data.py` to split the dataset into files for each party. 

The script allows specifying the number of parties as well as the dataset to use (from several supported datasets: _mnist_, _femnist_, _cifar10_ and many others). 

The `-pp` argument states how many data points to choose per party. If the option `--stratify` is given, the library stratifies the data proportionally according to the source distribution. If you want to run this notebook in different machines, you can assign samples for each party locally. Then, we define the neural network definition.

In [2]:
%run examples/generate_data.py -n $num_parties -d $dataset -pp 200 

Using TensorFlow backend.


Finished! :) Data saved in examples/data/adult/random


## Aggregator

The aggregator coordinates the overall process, communicates with the parties and integrates the results of the training process. This integration of results is done using the _Fusion Algorithm_.

A fusion algorithm queries the registered parties to carry out the federated learning process. The queries sent vary according to the model/algorithm type.  In return, parties send their reply as a model update object, and these model updates are then aggregated according to the specified Fusion Algorithm, specified via a `Fusion Handler` class. 

To take a look at the supported fusion algorithms, refer the IBM FL tutorial page [here](https://github.com/IBM/federated-learning-lib/blob/main/README.md#supported-functionality).

### Aggregator Configuration

We discuss the various configuration parameters for the Aggregator [here.](https://github.com/IBM/federated-learning-lib/blob/main/docs/tutorials/configure_fl.md#the-aggregators-configuration-file) Given below is an example of the aggregator's configuration file for the **Local Reweighing** method, which uses the **AdultSklearnDataHandler**, the **ReweighLocalTrainingHandler**, and the **IterAvgFusionHandler**.

To run the Global Reweighing with Differential Privacy method, we would use the **AdultSklearnDataHandler**, the **ReweighLocalTrainingHandler**, and the **ReweighFusionHandler**.

To run the Federated Prejudice Removal method, we would use the **AdultPRDataHandler**, the **PRLocalTrainingHandler**, the **SklearnPRFLModel**, and the **PrejRemoverFusionHandler**.

<img src="../images/arch_aggregator.png" width="680"/>
<figcaption><center>Image Source: <a href="https://arxiv.org/pdf/2007.10987.pdf">IBM Federated Learning: An Enterprise FrameworkWhite Paper V0.1</a></center></figcaption>

In [3]:
agg_config = {
    'connection': {
        'info': {
            'ip': '127.0.0.1',
            'port': 5000,
            'tls_config': {
                'enable': 'false'
            }
        },
        'name': 'FlaskConnection',
        'path': 'ibmfl.connection.flask_connection',
        'sync': 'False'
    },
    'data': {
        'info': {
            'txt_file': 'examples/datasets/adult.data'
        },
        'name': 'AdultSklearnDataHandler',
        'path': 'ibmfl.util.data_handlers.adult_sklearn_data_handler'
    },
    'fusion': {
        'name': 'IterAvgFusionHandler',
        'path': 'ibmfl.aggregator.fusion.iter_avg_fusion_handler'
    },
    'hyperparams': {
        'global': {
            'rounds': 3,
            'termination_accuracy': 0.9
        },
        'local': {
            'training': {
                'epochs': 3
            }
        }
    },
    'protocol_handler': {
        'name': 'ProtoHandler',
        'path': 'ibmfl.aggregator.protohandler.proto_handler'
    }
}

### Running the Aggregator
Next we pass the configuration parameters set in the previous cell to instantiate the `Aggregator` object. Finally, we `start()` the Aggregator process.

In [4]:

from ibmfl.aggregator.aggregator import Aggregator
aggregator = Aggregator(config_dict=agg_config)

aggregator.start()

2021-02-04 13:28:37,689 | 1.0.3 | INFO | ibmfl.util.config                             | Getting Aggregator details from arguments.
2021-02-04 13:28:38,248 | 1.0.3 | INFO | ibmfl.util.config                             | No model config provided for this setup.
2021-02-04 13:28:38,487 | 1.0.3 | INFO | ibmfl.util.config                             | No data config provided for this setup.
2021-02-04 13:28:38,487 | 1.0.3 | INFO | ibmfl.util.data_handlers.adult_sklearn_data_handler | Loaded training data from examples/datasets/adult.data


/Users/anniek@ibm.com/Desktop/ffl/ibmfl/util/data_handlers/adult_sklearn_data_handler.py:156: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  training_dataset['sex'] = training_dataset['sex'].map({' Female': 0, ' Male': 1})
/Users/anniek@ibm.com/Desktop/ffl/ibmfl/util/data_handlers/adult_sklearn_data_handler.py:158: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  {' Asian-Pac-Islander': 0, ' Amer-Indian-Eskimo': 0, ' Other': 0, ' Black': 0, ' White': 1})
/Users/anniek@ibm.com/Desktop/ffl/ibmfl/util/data_handlers/

2021-02-04 13:29:56,549 | 1.0.3 | INFO | ibmfl.connection.flask_connection             | RestSender initialized
2021-02-04 13:29:56,551 | 1.0.3 | INFO | ibmfl.aggregator.protohandler.proto_handler   | State: States.START
2021-02-04 13:29:56,551 | 1.0.3 | INFO | ibmfl.connection.flask_connection             | Receiver Initialized
2021-02-04 13:29:56,552 | 1.0.3 | INFO | ibmfl.connection.flask_connection             | Initializing Flask application
2021-02-04 13:29:56,556 | 1.0.3 | INFO | ibmfl.aggregator.aggregator                   | Aggregator initialization successful
2021-02-04 13:29:56,558 | 1.0.3 | INFO | ibmfl.aggregator.aggregator                   | Aggregator start successful
 * Serving Flask app "ibmfl.connection.flask_connection" (lazy loading)
 * Environment: production
   Use a production WSGI server instead.
 * Debug mode: off
2021-02-04 13:29:56,570 | 1.0.3 | INFO | werkzeug                                      |  * Running on http://127.0.0.1:5000/ (Press CTRL+C to quit

<img src="../images/arch_party.png" width="680"/>
<figcaption><center>Image Source: <a href="https://arxiv.org/pdf/2007.10987.pdf">IBM Federated Learning: An Enterprise FrameworkWhite Paper V0.1</a></center></figcaption>

## Starting Parties

Now that we have Aggregator running, next we go to Parties' notebooks (`sklearn_logclassification_p0.ipynb` and `sklearn_logclassification_p1.ipynb`) to start and register them with the Aggregator. Once all the parties are done with registration, we will move to next step to start training.

## Training and Evaluation

Now that our network has been set up, we begin training the model by invoking the Aggregator's `start_training()` method. 

This could take some time, depending on your system specifications. Feel free to get your doze of coffee meanwhile ☕

In [6]:
"""
#1 Initialize the metrics collector variables
"""
num_parties = 2
eval_party_accuracy = [[] for _ in range(2)]
iterations = [[] for _ in range(2)]

"""
#2 Register handler for metrics collector
"""
def get_metrics(metrics):
    keys = list(metrics['party'].keys())
    keys.sort()
    for i in range(len(keys)):
      eval_party_accuracy[i].append(metrics['party'][keys[i]]['acc'])
      iterations[i].append(metrics['fusion']['curr_round']*3)
      
mh = aggregator.fusion.metrics_manager
mh.register(get_metrics)


"""
#3 start the training
"""
aggregator.start_training()

2020-10-05 15:18:10,140 -STD ibmfl.aggregator.aggregator - INFO - Initiating Global Training.
2020-10-05 15:18:10,142 -STD ibmfl.aggregator.fusion.fusion_handler - INFO - Warm start disabled.
2020-10-05 15:18:10,144 -STD ibmfl.aggregator.fusion.iter_avg_fusion_handler - INFO - Model updateNone
2020-10-05 15:18:10,145 -STD ibmfl.aggregator.protohandler.proto_handler - INFO - State: States.SND_REQ
2020-10-05 15:18:10,258 -STD ibmfl.aggregator.protohandler.proto_handler - INFO - Total number of success responses :2
2020-10-05 15:18:10,260 -STD ibmfl.aggregator.protohandler.proto_handler - INFO - State: States.QUORUM_WAIT
2020-10-05 15:18:10,261 -STD ibmfl.aggregator.protohandler.proto_handler - INFO - Target Qorum: 2
2020-10-05 15:18:15,268 -STD ibmfl.aggregator.protohandler.proto_handler - INFO - Timeout:60 Time spent:5
2020-10-05 15:18:15,270 -STD ibmfl.aggregator.protohandler.proto_handler - INFO - Target Qorum: 2
2020-10-05 15:18:18,858 -STD ibmfl.connection.flask_connection - INFO - 

2020-10-05 15:19:11,878 -STD ibmfl.aggregator.fusion.iter_avg_fusion_handler - INFO - Model update<ibmfl.model.model_update.ModelUpdate object at 0x11966b128>
2020-10-05 15:19:11,881 -STD ibmfl.aggregator.protohandler.proto_handler - INFO - State: States.SND_REQ
2020-10-05 15:19:12,378 -STD ibmfl.aggregator.protohandler.proto_handler - INFO - Total number of success responses :2
2020-10-05 15:19:12,380 -STD ibmfl.aggregator.protohandler.proto_handler - INFO - State: States.QUORUM_WAIT
2020-10-05 15:19:12,383 -STD ibmfl.aggregator.protohandler.proto_handler - INFO - Target Qorum: 2
2020-10-05 15:19:17,390 -STD ibmfl.aggregator.protohandler.proto_handler - INFO - Timeout:60 Time spent:5
2020-10-05 15:19:17,392 -STD ibmfl.aggregator.protohandler.proto_handler - INFO - Target Qorum: 2
2020-10-05 15:19:22,399 -STD ibmfl.aggregator.protohandler.proto_handler - INFO - Timeout:60 Time spent:10
2020-10-05 15:19:22,402 -STD ibmfl.aggregator.protohandler.proto_handler - INFO - Target Qorum: 2
202

2020-10-05 15:20:14,231 -STD ibmfl.aggregator.protohandler.proto_handler - INFO - State: States.SND_REQ
2020-10-05 15:20:14,950 -STD ibmfl.aggregator.protohandler.proto_handler - INFO - Total number of success responses :2
2020-10-05 15:20:14,959 -STD ibmfl.aggregator.protohandler.proto_handler - INFO - State: States.QUORUM_WAIT
2020-10-05 15:20:14,969 -STD ibmfl.aggregator.protohandler.proto_handler - INFO - Target Qorum: 2
2020-10-05 15:20:19,974 -STD ibmfl.aggregator.protohandler.proto_handler - INFO - Timeout:60 Time spent:5
2020-10-05 15:20:19,977 -STD ibmfl.aggregator.protohandler.proto_handler - INFO - Target Qorum: 2
2020-10-05 15:20:24,986 -STD ibmfl.aggregator.protohandler.proto_handler - INFO - Timeout:60 Time spent:10
2020-10-05 15:20:24,989 -STD ibmfl.aggregator.protohandler.proto_handler - INFO - Target Qorum: 2
2020-10-05 15:20:29,999 -STD ibmfl.aggregator.protohandler.proto_handler - INFO - Timeout:60 Time spent:15
2020-10-05 15:20:30,002 -STD ibmfl.aggregator.protohand

2020-10-05 15:21:17,533 -STD ibmfl.aggregator.protohandler.proto_handler - INFO - Total number of success responses :2
2020-10-05 15:21:17,535 -STD ibmfl.aggregator.protohandler.proto_handler - INFO - State: States.QUORUM_WAIT
2020-10-05 15:21:17,536 -STD ibmfl.aggregator.protohandler.proto_handler - INFO - Target Qorum: 2
2020-10-05 15:21:22,543 -STD ibmfl.aggregator.protohandler.proto_handler - INFO - Timeout:60 Time spent:5
2020-10-05 15:21:22,545 -STD ibmfl.aggregator.protohandler.proto_handler - INFO - Target Qorum: 2
2020-10-05 15:21:24,013 -STD ibmfl.connection.flask_connection - INFO - Request received for path :7
2020-10-05 15:21:24,099 -STD werkzeug - INFO - 127.0.0.1 - - [05/Oct/2020 15:21:24] "POST /7 HTTP/1.1" 200 -
2020-10-05 15:21:24,162 -STD ibmfl.connection.flask_connection - INFO - Request received for path :7
2020-10-05 15:21:24,270 -STD werkzeug - INFO - 127.0.0.1 - - [05/Oct/2020 15:21:24] "POST /7 HTTP/1.1" 200 -
2020-10-05 15:21:27,551 -STD ibmfl.aggregator.proto

2020-10-05 15:22:08,752 -STD ibmfl.aggregator.protohandler.proto_handler - INFO - Timeout:60 Time spent:10
2020-10-05 15:22:08,754 -STD ibmfl.aggregator.protohandler.proto_handler - INFO - Target Qorum: 2
2020-10-05 15:22:08,755 -STD ibmfl.aggregator.protohandler.proto_handler - INFO - State: States.PROC_RSP
2020-10-05 15:22:08,766 -STD ibmfl.aggregator.fusion.iter_avg_fusion_handler - INFO - Model update<ibmfl.model.model_update.ModelUpdate object at 0x11b5f81d0>
2020-10-05 15:22:08,768 -STD ibmfl.aggregator.protohandler.proto_handler - INFO - State: States.SND_REQ
2020-10-05 15:22:09,196 -STD ibmfl.aggregator.protohandler.proto_handler - INFO - Total number of success responses :2
2020-10-05 15:22:09,197 -STD ibmfl.aggregator.protohandler.proto_handler - INFO - State: States.QUORUM_WAIT
2020-10-05 15:22:09,198 -STD ibmfl.aggregator.protohandler.proto_handler - INFO - Target Qorum: 2
2020-10-05 15:22:14,202 -STD ibmfl.aggregator.protohandler.proto_handler - INFO - Timeout:60 Time spen

2020-10-05 15:22:55,445 -STD ibmfl.aggregator.protohandler.proto_handler - INFO - Target Qorum: 2
2020-10-05 15:22:57,552 -STD ibmfl.connection.flask_connection - INFO - Request received for path :7
2020-10-05 15:22:57,556 -STD ibmfl.connection.flask_connection - INFO - Request received for path :7
2020-10-05 15:22:57,690 -STD werkzeug - INFO - 127.0.0.1 - - [05/Oct/2020 15:22:57] "POST /7 HTTP/1.1" 200 -
2020-10-05 15:22:57,758 -STD werkzeug - INFO - 127.0.0.1 - - [05/Oct/2020 15:22:57] "POST /7 HTTP/1.1" 200 -
2020-10-05 15:23:00,451 -STD ibmfl.aggregator.protohandler.proto_handler - INFO - Timeout:60 Time spent:10
2020-10-05 15:23:00,453 -STD ibmfl.aggregator.protohandler.proto_handler - INFO - Target Qorum: 2
2020-10-05 15:23:00,455 -STD ibmfl.aggregator.protohandler.proto_handler - INFO - State: States.PROC_RSP
2020-10-05 15:23:00,468 -STD ibmfl.aggregator.fusion.iter_avg_fusion_handler - INFO - Model update<ibmfl.model.model_update.ModelUpdate object at 0x10bcc0748>
2020-10-05 15

True

## Shut Down

Invoke the `stop()` method on each of the network participants to terminate the service.

In [8]:
aggregator.stop()

2020-10-05 15:23:59,072 -STD ibmfl.aggregator.protohandler.proto_handler - INFO - State: States.SND_REQ
2020-10-05 15:23:59,202 -STD ibmfl.aggregator.protohandler.proto_handler - INFO - Total number of success responses :2
2020-10-05 15:23:59,204 -STD ibmfl.connection.flask_connection - INFO - Stopping Receiver and Sender
2020-10-05 15:23:59,214 -STD werkzeug - INFO - 127.0.0.1 - - [05/Oct/2020 15:23:59] "POST /shutdown HTTP/1.1" 200 -
2020-10-05 15:23:59,219 -STD ibmfl.aggregator.aggregator - INFO - Aggregator stop successful


## Visualize Parties' Training
Please go to Parties' notebooks to visalize summary of Parties' training.